In [1]:
!pip install ipython-sql
!pip install ipython-sql prettytable

In [3]:
%load_ext sql

In [5]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [7]:
%sql sqlite:///my_data1.db

In [9]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [11]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [13]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [15]:
%sql SELECT DISTINCT Launch_Site FROM spacextable

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [19]:
%%sql SELECT *
FROM spacextable
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [32]:
%%sql select sum(PAYLOAD_MASS__KG_) from spacextable
where Customer like 'NASA (CRS)%';

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
48213


In [28]:
%%sql select avg(PAYLOAD_MASS__KG_) FROM SPACEXTABLE
WHERE Booster_Version like 'F9 v1.1%';

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2534.6666666666665


In [34]:
%%sql
SELECT MIN(Date) AS First_Successful_Landing_Date
FROM spacextable
WHERE Landing_Outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


First_Successful_Landing_Date
2015-12-22


In [36]:
%%sql
SELECT Booster_Version
FROM spacextable
WHERE Landing_Outcome = 'Success (drone ship)'
  AND Payload_Mass__KG_ > 4000
  AND Payload_Mass__KG_ < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [38]:
%%sql
SELECT 
    CASE
        WHEN Mission_Outcome LIKE 'Success%' THEN 'Success'
        WHEN Mission_Outcome LIKE 'Failure%' THEN 'Failure'
        ELSE 'Other'
    END AS Outcome_Category,
    COUNT(*) AS Total_Count
FROM spacextable
GROUP BY Outcome_Category;

 * sqlite:///my_data1.db
Done.


Outcome_Category,Total_Count
Failure,1
Success,100


In [40]:
%%sql
SELECT Booster_Version
FROM spacextable
WHERE Payload_Mass__KG_ = (
    SELECT MAX(Payload_Mass__KG_)
    FROM spacextable
);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [42]:
%%sql
SELECT 
    CASE 
        WHEN SUBSTR(Date, 6, 2) = '01' THEN 'January'
        WHEN SUBSTR(Date, 6, 2) = '02' THEN 'February'
        WHEN SUBSTR(Date, 6, 2) = '03' THEN 'March'
        WHEN SUBSTR(Date, 6, 2) = '04' THEN 'April'
        WHEN SUBSTR(Date, 6, 2) = '05' THEN 'May'
        WHEN SUBSTR(Date, 6, 2) = '06' THEN 'June'
        WHEN SUBSTR(Date, 6, 2) = '07' THEN 'July'
        WHEN SUBSTR(Date, 6, 2) = '08' THEN 'August'
        WHEN SUBSTR(Date, 6, 2) = '09' THEN 'September'
        WHEN SUBSTR(Date, 6, 2) = '10' THEN 'October'
        WHEN SUBSTR(Date, 6, 2) = '11' THEN 'November'
        WHEN SUBSTR(Date, 6, 2) = '12' THEN 'December'
    END AS Month_,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM spacextable
WHERE Landing_Outcome = 'Failure (drone ship)'
  AND SUBSTR(Date, 1, 4) = '2015';

 * sqlite:///my_data1.db
Done.


Month_,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [44]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM spacextable
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
